In [48]:
import pandas as pd
import allel
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [79]:
df_genes        = pd.read_csv('01-Apr-2022-GeneSummaries.csv')
df_variants     = pd.read_csv('01-Apr-2022-VariantSummaries.csv')
df_evidence     = pd.read_csv('01-Apr-2022-ClinicalEvidenceSummaries.csv')
df_assertions   = pd.read_csv('01-Apr-2022-AssertionSummaries.csv')
df_variantGroup = pd.read_csv('01-Apr-2022-VariantGroupSummaries.csv')
accepted_variants = allel.vcf_to_dataframe('01-Apr-2022-civic_accepted.vcf')
accepted_sub = allel.vcf_to_dataframe('01-Apr-2022-civic_accepted_and_submitted.vcf')

In [14]:
df_genes.head()

,gene_id,gene_civic_url,name,entrez_id,description,last_review_date,is_flagged
0,1,https://civic.genome.wustl.edu/links/genes/1,ALK,238,"ALK amplifications, fusions and mutations have...",2017-03-06 00:00:15 UTC,False
1,2,https://civic.genome.wustl.edu/links/genes/2,AKT1,207,"AKT1, also referred to as protein kinase B, is...",2017-02-09 21:58:06 UTC,False
2,3,https://civic.genome.wustl.edu/links/genes/3,ARAF,369,ARAF has recently become increasingly consider...,2015-06-21 16:49:19 UTC,False
3,4,https://civic.genome.wustl.edu/links/genes/4,ABL1,25,ABL1 is most relevant to cancer in its role in...,2015-06-21 16:49:20 UTC,False
4,5,https://civic.genome.wustl.edu/links/genes/5,BRAF,673,BRAF mutations are found to be recurrent in ma...,2019-09-19 19:32:44 UTC,False


### df_genes
 - gene_id = parece index mas de fato é o codigo do gene
 - gene_civic_url = url (está indo para a pagina onde possui todas as informaçoes do gene, incluindo sua descrição)
 - name = não possui nulo ou duplicado, aparentemente é o nome do gene (na descrição vários são citados)
 - entrez_id = não possui nulo ou duplicado, entender melhor que id é esse
 - description = apenas alguns possuem descrição, é a descrição do gene e suas características.(importante para interpretar as predições)
 - last_review_date = acredito ser irrelevante para o treinamente do modelo.
 - is_flagged = todos estão como false, facil detecção.

In [77]:
df_variants.head(2)

,variant_id,variant_civic_url,gene,entrez_id,variant,summary,variant_groups,chromosome,start,stop,reference_bases,variant_bases,representative_transcript,ensembl_version,reference_build,chromosome2,start2,stop2,representative_transcript2,variant_types,hgvs_expressions,last_review_date,civic_variant_evidence_score,allele_registry_id,clinvar_ids,variant_aliases,assertion_ids,assertion_civic_urls,is_flagged,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,1,https://civic.genome.wustl.edu/links/variants/1,ABL1,25,BCR::ABL,"The BCR-ABL fusion protein, commonly referred ...",ABL1 fusions in B-ALL,22,23522397.0,23632600.0,NaN,NaN,ENST00000305877.8,75.0,GRCh37,9,133729451.0,133763063.0,ENST00000318560.5,transcript_fusion,NaN,2022-03-10 22:27:25 UTC,255.5,NaN,NaN,"T(9;22)(Q34;Q11),BCR-ABL1,BCR-ABL",NaN,NaN,FALSE,NaN,NaN,NaN,NaN
1,2,https://civic.genome.wustl.edu/links/variants/2,ABL1,25,BCR::ABL T315I,While the efficacy of imatinib has revolutioni...,Imatinib Resistance,9,133748283.0,133748283.0,C,T,ENST00000318560.5,75.0,GRCh37,NaN,NaN,NaN,NaN,"missense_variant,transcript_fusion","NM_007313.2:c.1001C>T,NP_005148.2:p.Thr315Ile,...",2022-03-10 19:55:19 UTC,146.0,CA122575,12624,"THR334ILE,RS121913459,BCR-ABL THR315ILE,BCR-AB...",NaN,NaN,FALSE,NaN,NaN,NaN,NaN


### df_variants
 - variant_id = index
 - variant_civic_url = o mesmo que ocorre com o genes, detalhes da variante
 - gene = aqui os genes se repetem, pois mais de uma variante pode pertencer ao mesmo gene.
 - entrez_id = possui essa informação na tabela gene, possivel chave de ligação.
 - variant = é a variante do gene, assim como o gene pode possuir varias variantes o contrário também acontece.
 - summary = descrição e associação da variante em resposta ao câncer (importante para interpretar as predições)
 - variant_groups = preciso entender o df que consta no outro df
 - chromosome = poucos valores unicos e alguns nulos (em pesquisa me parece que cada cromossomo tem sua função, necessário entender melhor sobre)
 - start / stop
  
  
quando entro em detalhes para descobrir o start e stop, sou direcionada para esta pagina
está relacionado com a coluna anterior que é o cromossomo, no link de detalhe aponta como cordenada.
http://www.ensembl.org/Homo_sapiens/Transcript/Summary?g=ENSG00000186716;r=22:23180509-23318037;t=ENST00000305877

 - reference_bases = DNA, Adenina, Timina, Citosina e Guanina (está junto as coordenadas do cromossomo)
 - variant_bases = também sobre sequenciamento de DNA (também está posicionado em coordenadas)
 - representative_transcript = é o número da trasncrição das coordenadas (exemplo de link no star e stop)
 - ensembl_version = temos apenas 3 valores e 2 deles não se repetem, não sei ao certo o que significa, se é a versão do teste talvez, ou da transcrição...?
 - reference_build = está relacionada a coluna anterior, possui apenas 2 valores.
 - colunas irão se repetir, com valores diferentes, acredito que está associado a um novo teste de variante (colunas: chromosome2, start2, stop2 e representative_transcript2) os nulos aumentam nessa segunda, talvez menos genes foram testados mais de uma vez?
 - variant_types = nem todas as colunas possuem valores e as que possuem se repete, o gene/variante que possui pode receber varias informaçoes dentro dessa coluna (sao varias informações sobre variantes separadas por ',')
 - hgvs_expressions = São uma espécie de códigos que aparecem no detalhe na variante, 'representative_transcript' faz parte dele também, nem todos possuem
 - last_review_date = me parece irrelevante assim como no df de genes
 - civic_variant_evidence_score = pontuação do perfil molecular, possui varios duplicados mas nenhum nulo
 - allele_registry_id = ID de registro de alelo (Os alelos são as formas alternativas, gerados por mutações, de um mesmo gene), nem todos possuem
 - clinvar_ids = ClinVar ID possui um código associado a um link (https://www.ncbi.nlm.nih.gov/clinvar/variation/12624/?new_evidence=true) com varias informações genéticas, varios nulos.
 - variant_aliases = mais uma vez vários nomes de variantes juntos, separados por virgula, varios nulos
 - os 2 abaixo são relacionados, quando acessado o link, aparece um diagnóstico.
 - assertion_ids = são valores únicos, apenas 31 valores são nulos, são codigos numericos, ao acessar o link o código faz conjunto com a sigla 'AID', exemploa abaixo 'AID20'
 - assertion_civic_urls = link de url para detalhes do diagnosticos, também possui 31 valores nulos
https://civicdb.org/assertions/20/summary
 - is_flagged = facil detecção, não possui nulo, maioria FALSE, 1 TRUE, e o resto possui link do assertion(https://civicdb.org/assertions/10/summary)
 - As ultimas 4 colunas (Unnamed: 29	Unnamed: 30	Unnamed: 31	Unnamed: 32), possuem apenas 10 linhas com valores, sendo elas 'FALSE' ou o link do assertion (https://civicdb.org/assertions/12/summary), maioria com valores nulos.

In [132]:
df_evidence.head(2)

,gene,entrez_id,variant,disease,doid,phenotypes,drugs,drug_interaction_type,evidence_type,evidence_direction,evidence_level,clinical_significance,evidence_statement,citation_id,source_type,asco_abstract_id,citation,nct_ids,rating,evidence_status,evidence_id,variant_id,gene_id,chromosome,start,stop,reference_bases,variant_bases,representative_transcript,chromosome2,start2,stop2,representative_transcript2,ensembl_version,reference_build,variant_summary,variant_origin,last_review_date,evidence_civic_url,variant_civic_url,gene_civic_url,is_flagged
0,JAK2,3717,V617F,Lymphoid Leukemia,1037.0,NaN,NaN,NaN,Diagnostic,Supports,B,Negative,JAK2 V617F is not associated with lymphoid leu...,16081687,PubMed,NaN,"Levine et al., 2005, Blood",NaN,4.0,accepted,1,64,28,9,5073770.0,5073770.0,G,T,ENST00000381652.3,NaN,NaN,NaN,NaN,75.0,GRCh37,JAK2 V617F is a highly recurrent mutation in m...,Somatic,2020-11-20 16:18:21 UTC,https://civic.genome.wustl.edu/links/evidence_...,https://civic.genome.wustl.edu/links/variants/64,https://civic.genome.wustl.edu/links/genes/28,False
1,PDGFRA,5156,D842V,Gastrointestinal Stromal Tumor,9253.0,NaN,NaN,NaN,Diagnostic,Supports,B,Negative,GIST tumors harboring PDGFRA D842V mutation ar...,15146165,PubMed,NaN,"Lasota et al., 2004, Lab. Invest.",NaN,3.0,accepted,2,99,38,4,55152093.0,55152093.0,A,T,ENST00000257290.5,NaN,NaN,NaN,NaN,75.0,GRCh37,PDGFRA D842 mutations are characterized broadl...,Somatic,2015-06-21 16:49:38 UTC,https://civic.genome.wustl.edu/links/evidence_...,https://civic.genome.wustl.edu/links/variants/99,https://civic.genome.wustl.edu/links/genes/38,False


### df_evidence
 - gene = nenhuma valor nulo, alguns se repetem mas não possui duplicado
 - entrez_id = id aparece na tabela gene e variant
 - variant = não possui valor nulo, é o mesmo valor que aparece no df_variants['variant']
 - disease = diagnostico, varios duplicados, apenas 23 nulos. (possivel alvo preditor)
 - doid = possui 32 nulos, id de disease.
 - phenotypes = também tem relação com diagnóstico, mas voltado a carascteristicas, grande maioria nulo.
 - drugs = medicação, grande parte não possui informação.
 - drug_interaction_type = tipo de medicação, grande parte possui informação e somente 3 opções.
 - evidence_type = (possivel alvo preditor), 6 variaveis e inclui possivel diagnostico, não possui valores nulos
 - evidence_direction = poucos nulos, se suportou ou não o medicamento.
 - evidence_level = niveis de A a E ( “A - Validated association”, “B - Clinical evidence”, “C - Case study”, “D - Preclinical evidence”, and “E - Inferential association”. ), nào possui nulos
 - clinical_significance - acredito ser a resposta clinica ao tratamento, poucos nulos
 - evidence_statement = explicando a variante para o resultado. nenhum nulo
 - citation_id = id da publicação
 - source_type = nenhum nulo, onde foi publicado
 - asco_abstract_id = quase todos nulos, registro da publicacao no asco.
 - citation = local da publicação
 - nct_ids = quase todos nulos, ID do ensaio clínico associado ao item de evidência
 - rating = poucos nulos, nota para o estudo
 - evidence_status = todos aceitos
 - evidence_id = numero de registro da evidencia, todos unicos
 - variant_id = id da variante
 - gene_id = id do gene
 - chromosome = em pesquisa me parece que cada cromossomo tem sua função, necessário entender melhor sobre
 
 abaixo, todos já respondidos anteriormente
 - start = 
 - stop = 
 - reference_bases = 
 - variant_bases = 
 - representative_transcript = 
 - chromosome2 = 
 - start2 = 
 - stop2 = 
 - representative_transcript2 = 
 - ensembl_version =
 - reference_build =
 
 
 - variant_summary = muitos nulos, descrição da variante
 - variant_origin = como começou
 - last_review_date = irrelevante, será excluido
 - evidence_civic_url = link para a evidencia
 - variant_civic_url = link para a variante
 - gene_civic_url = link para o gene
 - is_flagged = facil detecção


In [145]:
df_assertions.head(2)

,gene,entrez_id,variant,disease,doid,phenotypes,drugs,assertion_type,assertion_direction,clinical_significance,acmg_codes,amp_category,nccn_guideline,nccn_guideline_version,regulatory_approval,fda_companion_test,assertion_summary,assertion_description,assertion_id,evidence_item_ids,variant_id,gene_id,last_review_date,assertion_civic_url,evidence_items_civic_url,variant_civic_url,gene_civic_url,is_flagged
0,ERBB2,2064,Amplification,Her2-receptor Positive Breast Cancer,60079.0,NaN,Trastuzumab,Predictive,Supports,Sensitivity/Response,NaN,Tier I - Level A,Breast Cancer,3.2017,True,True,HER2 amplification predicts sensitivity to Tra...,HER2 amplification defines a clinically releva...,2,"529,528,1122",18,20,2020-01-16 19:05:22 UTC,https://civic.genome.wustl.edu/links/assertions/2,https://civic.genome.wustl.edu/links/evidence_...,https://civic.genome.wustl.edu/links/variants/18,https://civic.genome.wustl.edu/links/genes/20,False
1,ALK,238,Fusion,Lung Non-small Cell Carcinoma,3908.0,NaN,Crizotinib,Predictive,Supports,Sensitivity/Response,NaN,Tier I - Level A,Non-Small Cell Lung Cancer,9.2017,True,True,Lung adenocarcinoma positive for ALK-FUSIONS h...,ALK-FUSIONS induce an active form of the recep...,3,"1419,1201,1200,4858,1199,1198,1356,1691,1187,1...",499,1,2020-01-16 19:05:22 UTC,https://civic.genome.wustl.edu/links/assertions/3,https://civic.genome.wustl.edu/links/evidence_...,https://civic.genome.wustl.edu/links/variants/499,https://civic.genome.wustl.edu/links/genes/1,False


### df_assertions

 - gene = nome do gene, nenhum nulo mas alguns se repetem
 - entrez_id = aparece em todas as tabelas anteriores, nenhum nulo mas alguns se repetem
 - variant = nome da variante, nenhum nulo mas alguns se repetem
 - disease = diagnóstico, nenhum nulo mas alguns se repetem
 - doid = id de disease, 1 nulo
 - phenotypes = também tem relação com diagnóstico, mas voltado a carascteristicas, grande maioria nulo.
 - drugs = medicação
 - assertion_type = (possivel alvo preditor), 6 variaveis e inclui possivel diagnostico, não possui valores nulos
 - assertion_direction = se suportou ou não o medicamento.
 - clinical_significance = se suportou ou não o medicamento.
 - acmg_codes = código clínico
 - amp_category = apelido na medicação e nível
 - nccn_guideline = tipo do cancer
 - nccn_guideline_version = id do cancer
 - regulatory_approval = se o tratamento aprovado, todos preenchidos
 - fda_companion_test = aprovação do fds (tipo anvisa)
 - assertion_summary = breve descrição do tratamento e diagnostico
 - assertion_description = descrição do tratamento e diagnostico
 - assertion_id = id do assertion
 - evidence_item_ids = ids de evidence, mais de 1 (provavelmente este é o teste do treinamento)
 - variant_id = id da variante
 - gene_id = id do gene
 - last_review_date = irrelevante
 - assertion_civic_url = link para o assertion
 - evidence_items_civic_url = link para evidence
 - variant_civic_url = link para variant
 - gene_civic_url = link para gene
 - is_flagged = se deu sinais positivos ou nao

In [144]:
df_variantGroup.head(3)

,variant_group_id,variant_group_civic_url,variant_group,description,last_review_date,is_flagged
0,1,https://civic.genome.wustl.edu/links/variant_g...,Imatinib Resistance,While imatinib has shown to be incredibly succ...,2015-06-21 16:49:55 UTC,False
1,2,https://civic.genome.wustl.edu/links/variant_g...,KIT Exon 17,NaN,2016-04-15 17:15:01 UTC,False
2,3,https://civic.genome.wustl.edu/links/variant_g...,Crizotinib Resistance,The ALK oncogene has long been considered a dr...,2015-06-21 16:49:55 UTC,False


### df_variantGroup
 - variant_group_id = index
 - variant_group_civic_url = possui link associado ao tratamento, nenhum valor nulo
 - variant_group = todos valores unicos e nenhum nulo, nome do tratamento
 - description = descrição do tratamento, alguns nulos
 - last_review_date = me parece irrelevante assim como no df de genes
 - is_flagged = não possui nulo e todos os valores são 'FALSE'

In [152]:
accepted_variants.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,1,8073509,691,C,A,NaN,NaN,NaN,False
1,1,11177096,609,C,A,NaN,NaN,NaN,False
2,1,11184573,542,G,T,NaN,NaN,NaN,False
3,1,11184580,284,G,A,NaN,NaN,NaN,False
4,1,11187094,470,G,T,NaN,NaN,NaN,False


In [5]:
accepted_sub.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,1,8073509,691,C,A,NaN,NaN,NaN,False
1,1,11177096,609,C,A,NaN,NaN,NaN,False
2,1,11184573,542,G,T,NaN,NaN,NaN,False
3,1,11184580,284,G,A,NaN,NaN,NaN,False
4,1,11187094,470,G,T,NaN,NaN,NaN,False


**acredito ser irrelevante**

In [232]:
#juntando dados do df variant em evidence
df_merge = df_evidence.merge(df_variants, left_on = 'variant_id', right_on = 'variant_id',
                 how='left', suffixes=('', '_y'))

df_merge.drop(df_merge.filter(regex='_y$').columns, axis=1, inplace=True)

In [229]:
''''
apenas se quiser adicionar a descrição dos genes, não possui para todos, para isso, necessário alterar nome da 
coluna da descrição para entendimento, assim como coluna name já possui no fd_evidence como coluna gene.

por se tratar de dados sem relevancia optei por nao ter esse trabalho
'''
#df_merge = df_merge.merge(df_genes, left_on = 'gene_id', right_on = 'gene_id',
#                 how='left', suffixes=('', '_y'))

#df_merge.drop(df_merge.filter(regex='_y$').columns, axis=1, inplace=True)

In [230]:
print(df_evidence.shape[0])
print(df_merge.shape[0])

3897
3897


In [231]:
df_merge.columns

Index(['gene', 'entrez_id', 'variant', 'disease', 'doid', 'phenotypes',
       'drugs', 'drug_interaction_type', 'evidence_type', 'evidence_direction',
       'evidence_level', 'clinical_significance', 'evidence_statement',
       'citation_id', 'source_type', 'asco_abstract_id', 'citation', 'nct_ids',
       'rating', 'evidence_status', 'evidence_id', 'variant_id', 'gene_id',
       'chromosome', 'start', 'stop', 'reference_bases', 'variant_bases',
       'representative_transcript', 'chromosome2', 'start2', 'stop2',
       'representative_transcript2', 'ensembl_version', 'reference_build',
       'variant_summary', 'variant_origin', 'last_review_date',
       'evidence_civic_url', 'variant_civic_url', 'gene_civic_url',
       'is_flagged', 'summary', 'variant_groups', 'variant_types',
       'hgvs_expressions', 'civic_variant_evidence_score',
       'allele_registry_id', 'clinvar_ids', 'variant_aliases', 'assertion_ids',
       'assertion_civic_urls', 'Unnamed: 29', 'Unnamed: 30', 'U